<a href="https://colab.research.google.com/github/polydeuces32/s-p500-predicter-on-Python-/blob/main/OPTIONS_CHART_PLAY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 #Install dependencies
!pip install yfinance ipywidgets --quiet

# Imports
import yfinance as yf
import numpy as np
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Greeks calculator ---
def calculate_greeks(S, K, T, r, sigma, option_type='call'):
    try:
        d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
        d2 = d1 - sigma * np.sqrt(T)
    except ZeroDivisionError:
        return 0, 0, 0, 0, 0

    if option_type == 'call':
        delta = norm.cdf(d1)
        rho = K * T * np.exp(-r * T) * norm.cdf(d2) / 100
    else:
        delta = -norm.cdf(-d1)
        rho = -K * T * np.exp(-r * T) * norm.cdf(-d2) / 100

    gamma = norm.pdf(d1) / (S * sigma * np.sqrt(T))
    theta = (
        -S * norm.pdf(d1) * sigma / (2 * np.sqrt(T))
        - r * K * np.exp(-r * T) * norm.cdf(d2 if option_type == 'call' else -d2)
    )
    vega = S * norm.pdf(d1) * np.sqrt(T) / 100

    return delta, gamma, theta, vega, rho

# --- Estimate trade quality ---
def estimate_trade_quality(delta, vega, avg_vega):
    delta_score = abs(delta)
    vega_score = 1 - abs(vega - avg_vega)
    return round(100 * (0.5 * delta_score + 0.3 * vega_score), 2)  # % score

# --- Put-call parity ---
def put_call_parity(S, K, T, r, call_price, put_price):
    theo_put = call_price + K * np.exp(-r * T) - S
    diff = put_price - theo_put
    return theo_put, diff

# --- Widgets ---
ticker_input = widgets.Text(value='AAPL', description='Ticker:')
option_type_dropdown = widgets.Dropdown(options=['call', 'put'], description='Type:')
moneyness_filter = widgets.Dropdown(options=['all', 'ITM', 'ATM', 'OTM'], description='Filter:')
expiration_dropdown = widgets.Dropdown(options=[], description='Expiration:')
run_button = widgets.Button(description="Run Analysis", button_style='success')
output = widgets.Output()

ui = widgets.VBox([ticker_input, option_type_dropdown, moneyness_filter, expiration_dropdown, run_button])
display(ui, output)

# --- Update Expirations ---
def update_expiration_dropdown(*args):
    try:
        ticker = yf.Ticker(ticker_input.value.upper())
        expiration_dropdown.options = ticker.options
    except:
        expiration_dropdown.options = []

ticker_input.observe(update_expiration_dropdown, names='value')
update_expiration_dropdown()

# --- Main logic ---
def on_run_clicked(b):
    with output:
        clear_output()
        symbol = ticker_input.value.upper()
        option_type = option_type_dropdown.value
        m_filter = moneyness_filter.value
        expiration = expiration_dropdown.value

        try:
            ticker = yf.Ticker(symbol)
            stock_price = ticker.history(period="1d")['Close'].iloc[-1]
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return

        print(f"Live price for {symbol}: ${stock_price:.2f}")
        print(f"Selected expiration: {expiration}")
        T = (pd.to_datetime(expiration) - datetime.today()).days / 365
        r = 0.02

        chain = ticker.option_chain(expiration)
        options_df = chain.calls if option_type == 'call' else chain.puts
        options_df = options_df[options_df['impliedVolatility'] > 0]

        # Moneyness filtering
        if m_filter != 'all':
            if option_type == 'call':
                if m_filter == 'ITM':
                    options_df = options_df[options_df['strike'] < stock_price]
                elif m_filter == 'OTM':
                    options_df = options_df[options_df['strike'] > stock_price]
                else:
                    idx = options_df['strike'].sub(stock_price).abs().idxmin()
                    options_df = options_df.loc[[idx]]
            else:
                if m_filter == 'ITM':
                    options_df = options_df[options_df['strike'] > stock_price]
                elif m_filter == 'OTM':
                    options_df = options_df[options_df['strike'] < stock_price]
                else:
                    idx = options_df['strike'].sub(stock_price).abs().idxmin()
                    options_df = options_df.loc[[idx]]

        # Greeks & Trade Score
        greeks_data = []
        vega_list = []
        for _, row in options_df.iterrows():
            K = row['strike']
            sigma = row['impliedVolatility']
            delta, gamma, theta, vega, rho = calculate_greeks(stock_price, K, T, r, sigma, option_type)
            vega_list.append(vega)
            greeks_data.append([K, delta, gamma, theta, vega, rho, row['lastPrice']])

        avg_vega = np.mean(vega_list)
        data = []
        for i, row in enumerate(greeks_data):
            quality = estimate_trade_quality(row[1], row[4], avg_vega)
            data.append({
                "Strike": row[0],
                "Delta": row[1],
                "Gamma": row[2],
                "Theta": row[3],
                "Vega": row[4],
                "Rho": row[5],
                "Price": row[6],
                "Trade Quality (%)": quality
            })

        df = pd.DataFrame(data).sort_values("Strike")
        display(df)

        # Plot Greeks
        plt.figure(figsize=(12, 6))
        for col in ['Delta', 'Gamma', 'Theta', 'Vega', 'Rho']:
            plt.plot(df['Strike'], df[col], label=col)
        plt.title(f"{symbol} {option_type.upper()} Greeks - Exp: {expiration}")
        plt.xlabel("Strike Price")
        plt.ylabel("Value")
        plt.legend()
        plt.grid(True)
        plt.show()

        # Put-call parity check
        if option_type == 'call':
            puts_df = chain.puts
            puts_df = puts_df[puts_df['strike'].isin(df['Strike'])]
            parity = []
            for i in range(len(df)):
                K = df.iloc[i]['Strike']
                call_price = df.iloc[i]['Price']
                try:
                    put_price = puts_df[puts_df['strike'] == K]['lastPrice'].values[0]
                    theo_put, diff = put_call_parity(stock_price, K, T, r, call_price, put_price)
                    parity.append([K, put_price, theo_put, diff])
                except:
                    continue
            if parity:
                parity_df = pd.DataFrame(parity, columns=["Strike", "Put Price", "Theoretical Put", "Diff"])
                print("\n📉 Put-Call Parity Check:")
                display(parity_df)

# Bind
run_button.on_click(on_run_clicked)
